In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
PPT → Markdown 변환기 (v4: 타이틀 추출 개선 + 위치 기반 정렬)

개선 포인트 (v4):
 - 타이틀 추출 개선: placeholder type 확인 (TITLE, CENTER_TITLE)
 - Shape 위치 기반 정렬: 상단→하단, 좌측→우측 순서로 추출
 - 타이틀과 본문 분리: 타이틀로 인식된 shape는 본문에서 제외
 - 테이블 셀 줄바꿈: \n → <br/> 변환하여 테이블 깨짐 방지
 - 테이블 중복 제거: 테이블 shape는 텍스트 추출에서 제외

기존 v3 기능:
 - paragraph 수준으로 run을 병합하여 문장 단위 복원
 - a:br(줄바꿈) 반영, 공백 정리, soft hyphen 제거
 - 기본 bullet 접두사 처리
 - 이미지/테이블/수식/헤더 추출

사용법:
 python ppt_to_markdown_v4.py "ppt/1-2 데이터의 정제.pptx" "1-2 데이터의 정제.ipynb"
"""

import os
import mimetypes
import base64
import zipfile
import re
from typing import List, Dict, Optional
from xml.etree import ElementTree as ET
import nbformat
from nbformat.v4 import new_notebook, new_markdown_cell

def _safe_text(s: Optional[str]) -> str:
    if type(s) != str:
        return ""
    return (s or "").strip()

def _smooth_join(parts: List[str]) -> str:
    buf, prev = [], ""
    for cur in parts:
        if cur is None:
            continue
        cur = cur.replace("\u00ad", "")
        if not prev:
            buf.append(cur)
        else:
            need_space = False
            if prev and cur:
                if prev[-1].isalnum() and cur[0].isalnum():
                    need_space = True
                if cur[0] in ".,;:!?)]}":
                    need_space = False
                if prev[-1].isspace() or cur[0].isspace():
                    need_space = False
            if need_space:
                buf.append(" ")
            buf.append(cur)
        prev = cur
    text = "".join(buf)
    text = re.sub(r"\s+([,.;:!?])", r"\1", text)
    text = re.sub(r"\s{2,}", " ", text)
    return text

def _image_to_data_uri(data: bytes, ext: str) -> str:
    ctype = mimetypes.types_map.get(ext.lower(), "application/octet-stream")
    b64 = base64.b64encode(data).decode("ascii")
    return f"data:{ctype};base64,{b64}"

def _collect_with_python_pptx(pptx_path: str) -> List[Dict]:
    try:
        from pptx import Presentation
        from pptx.enum.shapes import MSO_SHAPE_TYPE, PP_PLACEHOLDER
    except Exception:
        return []
    prs = Presentation(pptx_path)
    slides_out = []
    for idx, slide in enumerate(prs.slides):
        # 타이틀 추출 개선: placeholder type 확인
        title_text = None
        title_shape = None

        # 1차: placeholder type으로 타이틀 찾기
        for shp in slide.shapes:
            if shp.is_placeholder:
                try:
                    phf = shp.placeholder_format
                    # TITLE (1), CENTER_TITLE (3)
                    if phf.type in (1, 3):
                        if hasattr(shp, "text_frame") and shp.text_frame:
                            text = _safe_text(shp.text_frame.text)
                            if text:
                                title_text = text
                                title_shape = shp
                                break
                except Exception:
                    pass

        # 2차: shape 이름이 "Title"로 시작하는 것 찾기
        if not title_text:
            for shp in slide.shapes:
                if shp.name and shp.name.startswith("Title"):
                    if getattr(shp, "has_text_frame", False) and shp.has_text_frame:
                        text = _safe_text(shp.text_frame.text)
                        if text:
                            title_text = text
                            title_shape = shp
                            break

        # 3차: 가장 위쪽에 있는 텍스트를 타이틀로
        if not title_text:
            text_shapes = []
            for shp in slide.shapes:
                if getattr(shp, "has_text_frame", False) and shp.has_text_frame:
                    text = _safe_text(shp.text_frame.text)
                    if text:
                        text_shapes.append((shp.top, shp, text))
            if text_shapes:
                text_shapes.sort(key=lambda x: x[0])  # top 기준 정렬
                title_text = text_shapes[0][2][:120]
                title_shape = text_shapes[0][1]

        # Shape들을 위치 순서로 정렬 (상단→하단, 좌측→우측)
        sorted_shapes = sorted(
            slide.shapes,
            key=lambda shp: (shp.top, shp.left)
        )

        paragraphs, images, tables = [], [], []
        for shp in sorted_shapes:
            # 타이틀은 제외 (이미 추출함)
            if shp is title_shape:
                continue

            # 테이블 shape는 텍스트 추출에서 제외 (중복 방지)
            has_table = getattr(shp, "has_table", False)
            
            if not has_table and getattr(shp, "has_text_frame", False) and shp.has_text_frame:
                for p in shp.text_frame.paragraphs:
                    runs_text = [r.text or "" for r in p.runs]
                    para = _smooth_join(runs_text)
                    if not para:
                        continue
                    try:
                        if p.level is not None and getattr(p, "bullet", None) is not None:
                            indent = "  " * int(p.level)
                            para = f"{indent}- {para}"
                    except Exception:
                        pass
                    paragraphs.append(para)
            if getattr(shp, "shape_type", None) == MSO_SHAPE_TYPE.PICTURE:
                try:
                    img = shp.image
                    images.append({"data": img.blob, "ext": getattr(img, "ext", None) or ".png"})
                except Exception:
                    pass
            if has_table:
                try:
                    tbl = shp.table
                    t_rows = []
                    for r in tbl.rows:
                        row_vals = []
                        for c in r.cells:
                            cell_lines = []
                            for pp in c.text_frame.paragraphs:
                                runs_text = [rr.text or "" for rr in pp.runs]
                                cell_line = _smooth_join(runs_text)
                                if cell_line:
                                    cell_lines.append(cell_line)
                            # 셀 내용의 줄바꿈을 <br/>로 변환
                            row_vals.append("<br/>".join(cell_lines))
                        t_rows.append(row_vals)
                    if t_rows:
                        tables.append(t_rows)
                except Exception:
                    pass
        slides_out.append({
            "index": idx,
            "title": title_text or "",
            "paragraphs": paragraphs,
            "images": images,
            "tables": tables,
            "equations": []
        })
    return slides_out

def _collect_with_zip(pptx_path: str) -> List[Dict]:
    slides_out = []
    with zipfile.ZipFile(pptx_path, "r") as zf:
        slide_names = sorted([n for n in zf.namelist() if n.startswith("ppt/slides/slide") and n.endswith(".xml")],
                             key=lambda x: int("".join(filter(str.isdigit, x)) or "0"))
        ns = {
            "a": "http://schemas.openxmlformats.org/drawingml/2006/main",
            "p": "http://schemas.openxmlformats.org/presentationml/2006/main",
            "r": "http://schemas.openxmlformats.org/officeDocument/2006/relationships",
            "m": "http://schemas.openxmlformats.org/officeDocument/2006/math",
        }
        for idx, slide_name in enumerate(slide_names):
            root = ET.fromstring(zf.read(slide_name))
            title_text = ""
            for sp in root.findall(".//p:sp", ns):
                p = sp.find(".//a:p", ns)
                if p is not None:
                    parts = []
                    for node in list(p):
                        if node.tag == f"{{{ns['a']}}}r":
                            t = node.find(".//a:t", ns)
                            parts.append(t.text if (t is not None and t.text) else "")
                        elif node.tag == f"{{{ns['a']}}}br":
                            parts.append("\n")
                    title_text = _safe_text(_smooth_join(parts)).splitlines()
                    if len(title_text) > 0:
                        title_text = title_text[0][:120]
                    if title_text:
                        break
            paragraphs = []
            for p in root.findall(".//a:p", ns):
                parts = []
                for node in list(p):
                    if node.tag == f"{{{ns['a']}}}r":
                        t = node.find(".//a:t", ns)
                        parts.append(t.text if (t is not None and t.text) else "")
                    elif node.tag == f"{{{ns['a']}}}br":
                        parts.append("\n")
                para = _safe_text(_smooth_join(parts))
                if para:
                    has_bullet = p.find(".//a:buChar", ns) is not None or p.find(".//a:buAutoNum", ns) is not None
                    if has_bullet:
                        para = f"- {para}"
                    paragraphs.append(para)
            images = []
            rels_name = slide_name.replace("slides/", "slides/_rels/") + ".rels"
            if rels_name in zf.namelist():
                rels_xml = ET.fromstring(zf.read(rels_name))
                rel_ns = {"r": "http://schemas.openxmlformats.org/package/2006/relationships"}
                for rel in rels_xml.findall(".//r:Relationship", rel_ns):
                    target = rel.attrib.get("Target", "")
                    rtype = rel.attrib.get("Type", "")
                    if "image" in rtype or target.startswith("../media"):
                        if target.startswith("../"):
                            media_path = "ppt/" + target.replace("../", "")
                        else:
                            media_path = "ppt/slides/" + target
                        if "ppt/media/" not in media_path:
                            candidate = "ppt/media/" + os.path.basename(target)
                            if candidate in zf.namelist():
                                media_path = candidate
                        if media_path in zf.namelist():
                            img_bytes = zf.read(media_path)
                            ext = os.path.splitext(media_path)[1] or ".png"
                            images.append({"data": img_bytes, "ext": ext})
            tables = []
            for tbl in root.findall(".//a:tbl", ns):
                rows = []
                for tr in tbl.findall(".//a:tr", ns):
                    row_vals = []
                    for tc in tr.findall("a:tc", ns):
                        cell_parts = []
                        for p in tc.findall(".//a:p", ns):
                            prts = []
                            for node in list(p):
                                if node.tag == f"{{{ns['a']}}}r":
                                    t = node.find(".//a:t", ns)
                                    prts.append(t.text if (t is not None and t.text) else "")
                                elif node.tag == f"{{{ns['a']}}}br":
                                    prts.append("\n")
                            line = _safe_text(_smooth_join(prts))
                            if line:
                                cell_parts.append(line)
                        # 셀 내용의 줄바꿈을 <br/>로 변환
                        row_vals.append("<br/>".join(cell_parts))
                    if row_vals:
                        rows.append(row_vals)
                if rows:
                    tables.append(rows)
            equations = []
            for mnode in root.findall(".//m:oMath", ns) + root.findall(".//m:oMathPara", ns):
                toks = [mt.text for mt in mnode.findall(".//m:t", ns) if mt.text]
                eq = _safe_text("".join(toks))
                if eq:
                    equations.append(eq)
            slides_out.append({
                "index": idx,
                "title": title_text,
                "paragraphs": paragraphs,
                "images": images,
                "tables": tables,
                "equations": equations
            })
    return slides_out

def collect_pptx_contents(pptx_path: str) -> List[Dict]:
    if not os.path.exists(pptx_path):
        raise FileNotFoundError(f"PPTX not found: {pptx_path}")
    slides = _collect_with_python_pptx(pptx_path)
    if slides:
        try:
            xml_slides = _collect_with_zip(pptx_path)
            for a, b in zip(slides, xml_slides):
                a["equations"] = b.get("equations", [])
        except Exception:
            pass
        return slides
    return _collect_with_zip(pptx_path)

def _table_to_markdown(rows: List[List[str]]) -> str:
    if not rows:
        return ""
    max_cols = max(len(r) for r in rows)
    norm = [r + [""] * (max_cols - len(r)) for r in rows]
    header = norm[0]
    body = norm[1:] if len(norm) > 1 else []
    md = []
    md.append("| " + " | ".join(h if h else " " for h in header) + " |")
    md.append("| " + " | ".join("---" for _ in header) + " |")
    for r in body:
        md.append("| " + " | ".join(c if c else " " for c in r) + " |")
    return "\n".join(md)

def slides_to_markdown_cells(slides: List[Dict]) -> List[str]:
    md_cells = []
    prev_title = None
    for i, s in enumerate(slides, start=1):
        title = _safe_text(s.get("title") or f"Slide {i}")
        if title and title != prev_title:
            md_cells.append(f"# {title}")
            prev_title = title
        body = [f"### Slide {i}: {title if title else '(Untitled)'}", ""]
        for par in s.get("paragraphs", []):
            body.append(f"- {par}" if not par.lstrip().startswith(("-", "*", "+")) else par)
        for tbl in s.get("tables", []):
            body.append("")
            body.append(_table_to_markdown(tbl))
        for k, eq in enumerate(s.get("equations", []), start=1):
            if not eq:
                continue
            body.append("")
            body.append(f"**Equation {i}.{k}**")
            body.append(f"$$\n{eq}\n$$")
        if s.get("images"):
            body.append("")
            for j, img in enumerate(s["images"], start=1):
                ext = img.get("ext") or ".png"
                data_uri = _image_to_data_uri(img.get("data", b""), ext)
                body.append(f"![slide{i}-image{j}]({data_uri})")
        md_cells.append("\n".join(body).strip() + "\n")
    return md_cells

def build_output_notebook(pptx_path: str, out_ipynb: str) -> str:
    """PPTX 파일을 Jupyter 노트북으로 변환"""
    slides = collect_pptx_contents(pptx_path)
    md_cells = slides_to_markdown_cells(slides)
    nb = new_notebook()
    header = f"""# {os.path.splitext(os.path.basename(pptx_path))[0]}

**Source**: `{os.path.basename(pptx_path)}`

개선 사항 (v4):
- **타이틀 추출 개선**: placeholder type 확인 (TITLE, CENTER_TITLE)
- **Shape 위치 기반 정렬**: 상단→하단, 좌측→우측 순서로 정확하게 추출
- **타이틀과 본문 분리**: 타이틀로 인식된 shape는 본문에서 제외
- **테이블 셀 줄바꿈**: \\n → <br/> 변환하여 테이블 깨짐 방지
- **테이블 중복 제거**: 테이블 shape는 텍스트 추출에서 제외

기존 기능 (v3):
- 폰트/스타일로 분리된 run들을 문장 단위로 매끄럽게 연결
- 줄바꿈(br) 보존, 불필요한 공백 정리, 소프트 하이픈 제거
- bullet(기본) 감지 후 접두사 적용
- 이미지/테이블/수식 추출
"""
    nb.cells.append(new_markdown_cell(header.strip()))
    for md in md_cells:
        nb.cells.append(new_markdown_cell(md))
    with open(out_ipynb, "w", encoding="utf-8") as f:
        nbformat.write(nb, f)
    return out_ipynb

In [3]:
# Default conversion
pptx_path = "ppt/4-3 경사하강법 심화.pptx"
out_ipynb = "4-3 경사하강법 심화.ipynb"
result = build_output_notebook(pptx_path, out_ipynb)
print(f"✓ 변환 완료: {result}")


✓ 변환 완료: 4-3 경사하강법 심화.ipynb
